In [ ]:
!pip install openai-whisper gtts sounddevice numpy IPython

In [ ]:
#mikrofon indexini belirleme
import sounddevice as sd

for idx, device in enumerate(sd.query_devices()):
    if device['max_input_channels']>0:
        print(f"{idx}: {device['name']} (input: {device['max_input_channels']})")

In [ ]:
#ses kaydı alma ve dinleme
import sounddevice as sd
from scipy.io.wavfile import write
from IPython.display import Audio
import numpy as np

#mikrofon indexi
sd.default.device=0

fs=16000
saniye=5

print("Kayda başlamak için konuşunuz...")
myrecording=sd.rec(int(saniye*fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()
print("kayıt tamam.")
write('recorded.wav',fs,myrecording)

#ses dosyasını oynat
display(Audio('recorded.wav'))

print("Max:",np.max(myrecording),"Min:",np.min(myrecording),"Mean:",np.mean(myrecording))

In [ ]:
#sesin metine çevirilmesi
import whisper

model=whisper.load_model("base")
result=model.transcribe("recorded.wav", language="tr")
print("Transkrip:",result["text"])

In [ ]:
#LM Studio ile yanıt üretme
import openai

client = openai.OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"
)

system_message = """
Sen bir Türkçe konuşan sesli asistansın.
Yanıtların kısa, net ve samimi olsun.
"""

def ask_llm(prompt, system_message=system_message):
    response = client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7,
    )
    return response.choices[0].message.content

llm_response=ask_llm(result["text"])
print("Asistan:", llm_response)

In [ ]:
#yanıtı sese çevirme
from gtts import gTTS
from IPython.display import Audio
tts=gTTS(text=llm_response, lang="tr")
tts.save("response.mp3")
display(Audio("response.mp3",autoplay=True))